# Задача
Ребята в тексте ссылка на статью по прогонозу с учетом сезонности и тренда. ( https://4analytics.ru/prognozirovanie/kak-rasschitat-prognoz-prodaj-s-uchetom-rosta-i-sezonnosti-v-excel.html ).

Ваша задача прочитать статью и потом на основе данных к прошлому ДЗ сделать прогноз ТО на 2018 по этой методологии

# Подготовка

## Подключение библиотек

In [1]:
from sshtunnel import SSHTunnelForwarder
import mysql.connector
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import getpass

## Запуск сервера

In [2]:
public_ip = '3.124.214.86'

In [3]:
password_mysql = getpass.getpass(prompt="Введите пароль от MySQL-сервера: ")

server = SSHTunnelForwarder(
    (public_ip, 22),
    ssh_host_key=None,
    ssh_username="ubuntu",
    ssh_password=None,
    ssh_private_key='../key1_GB.pem',
    ssh_private_key_password=None,
    remote_bind_address=("127.0.0.1", 3306))

server.start()

mydb=mysql.connector.connect(
    user='root',
    password=password_mysql,
    host="127.0.0.1", 
    port=server.local_bind_port, 
    database='db_for_analyt')

print(mydb)

Введите пароль от MySQL-сервера:  ········


In [4]:
mycursor = mydb.cursor()
current_db = "orders_filter"

In [5]:
def execute_(sql, columns):
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    return pd.DataFrame(myresult, columns=columns)

# Выполнение запросов

## Запросы

### Проверка по цене меньше или равно 1

In [74]:
%%time
execute_(
    sql=f"SELECT COUNT(*) FROM {current_db} WHERE price <= 0", columns=['отрицательные заказы'])

Wall time: 884 ms


,отрицательные заказы
0,0


In [75]:
mycursor.execute(f"DELETE LOW_PRIORITY QUICK FROM {current_db} WHERE price <= 10")
mydb.commit()
print(mycursor.rowcount, "record(s) deleted")

0 record(s) deleted


### Проверка по макс цене

In [76]:
%%time
max_price = f"SELECT id_o, price FROM {current_db} WHERE price > 300000"
execute_(
    sql=max_price, 
    columns=['id','price'])

Wall time: 963 ms


,id,price
0,2456333,616994.0
1,2524864,819096.6
2,4978851,521336.2
3,5597085,392205.1
4,6108272,540482.6


In [77]:
mycursor.execute(f"DELETE LOW_PRIORITY QUICK FROM {current_db} WHERE price > 300000")
mydb.commit()
print(mycursor.rowcount, "record(s) deleted")

5 record(s) deleted


### Проверка дублирования id_o

In [78]:
%%time
execute_(
    sql=f"SELECT COUNT(*), COUNT(DISTINCT id_o) FROM {current_db}", 
    columns=['id','уник id_o'])

Wall time: 2.25 s


,id,уник id_o
0,2002529,2002529


In [6]:
%%time
execute_(
    sql=f"SELECT COUNT(*) FROM {current_db}", columns=['отрицательные заказы'])

Wall time: 446 ms


,отрицательные заказы
0,2002529


### Проверка отсутствия дней без заказов

In [7]:
execute_(
    sql='SELECT COUNT(o_date) FROM ('
        'SELECT o_date, SUM(price) s, COUNT(id_o) FROM orders_filter GROUP BY o_date) AS filt', 
    columns=['днй, где есть заказы'])

,"днй, где есть заказы"
0,731


### ТО по месяцам

In [12]:
df = execute_(
    sql='SELECT YEAR(o_date),MONTH(o_date), CEIL(SUM(price)) FROM orders_filter '
    'GROUP BY YEAR(o_date), MONTH(o_date) '
        'ORDER BY YEAR(o_date)', 
    columns=['год', 'месяц', 'сумма ТО'])
df.head()

,год,месяц,сумма ТО
0,2016,1,96812326.0
1,2016,2,95436525.0
2,2016,3,115270552.0
3,2016,4,139323970.0
4,2016,5,108587353.0


In [14]:
df.to_excel('ТО по месяцам.xlsx')

# Закрытие сессии

In [15]:
mydb.close()

In [16]:
server.stop()